# **L&T Financial Services Data Science FinHack in Analytics Vidhya**

**Link to Competition: https://datahack.analyticsvidhya.com/contest/ltfs-datascience-finhack-an-online-hackathon/#About**

**Leaderboard Rank - #143**

https://datahack.analyticsvidhya.com/contest/ltfs-datascience-finhack-an-online-hackathon/#LeaderBoard

**Download Dataset**

https://www.kaggle.com/chithiraimeenal/ltfs-dataset

**About LTFS Data Science FinHack ( ML Hackathon)**

L&T Financial Services & Analytics Vidhya presents ‘DataScience FinHack’.

Amazing opportunity for all creative nerds to apply their data science & machine learning skillset to best solve a real business problem.

In this FinHack, you will develop a model for our most common but real challenge ‘Loan Default Prediction’ & also, get a feel of our business!

If your solution adds good value to our organization, take it from us, Sky is the limit for you!

**About L&T Financial Services (LTFS):**

Headquartered in Mumbai, LTFS is one of India’s most respected & leading NBFCs providing finance for two wheeler, farm equipment, housing, infra & microfinance. With a strong parentage & stable leadership, it also has a flourishing Mutual Fund & Wealth Advisory business under its broad umbrella.

**Objective:**

To predict the probabilty of a customer defaulting on his loans using Machine Learning

**Evaluation Criteria:**

AUC Score


In [1]:
#Importing Libraries
library(dplyr)
library(caret)
library(lightgbm)
library(data.table)
library(tidyverse)
library(stringr)
library(Matrix)
library(caTools)
library(lightgbm)
library(pROC)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: lattice
Loading required package: ggplot2
Warning message:
“replacing previous import ‘ggplot2::empty’ by ‘plyr::empty’ when loading ‘caret’”Loading required package: R6

Attaching package: ‘lightgbm’

The following object is masked from ‘package:dplyr’:

    slice


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  2.0.1     ✔ purrr   0.3.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ data.table::between() masks dplyr::between()
✖ dplyr::filter()       masks stats::filter()
✖ data.table::first()   masks 

**Data Ingestion & Pre-processing**

In [2]:
train_df <- fread("../input/ltfs-train/train.csv")
test_df <- fread("../input/lfts-test/test_bqCt9Pv.csv")

In [3]:
train_df$flag=0
test_df$flag=1

In [4]:
label = train_df$loan_default
class(label)

[1] "integer"

In [5]:
data = rbind(train_df[,-c("loan_default")], test_df)

In [6]:
data <- mutate_at(data, vars("UniqueID","branch_id","supplier_id","manufacturer_id","Current_pincode_ID","State_ID","Employee_code_ID","MobileNo_Avl_Flag",
                         "Aadhar_flag","PAN_flag","VoterID_flag","Driving_flag","Passport_flag","Employment.Type","PERFORM_CNS.SCORE.DESCRIPTION"), as.character)

data <- mutate_at(data, vars("UniqueID","branch_id","supplier_id","manufacturer_id","Current_pincode_ID","State_ID","Employee_code_ID","MobileNo_Avl_Flag",
                             "Aadhar_flag","PAN_flag","VoterID_flag","Driving_flag","Passport_flag","Employment.Type","PERFORM_CNS.SCORE.DESCRIPTION"), as.factor)


In [7]:
data$DisbursalDate<-as.Date(as.character(data$DisbursalDate), "%d-%m-%y")
data$Date.of.Birth<-as.Date(as.character(data$Date.of.Birth), "%d-%m-%y")
#prop.table(table(data$loan_default))

**Feature Engineering**

In [8]:
data$Age = round(as.numeric(difftime(data$DisbursalDate, data$Date.of.Birth, unit="weeks"))/52.25,4)

In [9]:
#data$Year_Disbursed <- format(data$DisbursalDate,"%Y")
data$Month_Disbursed <- format(data$DisbursalDate,"%m")
data$Day_Disbursed <- format(data$DisbursalDate,"%d")
data$Week_Disbursed <- format(data$DisbursalDate,"%U")
data$Weekday_Disbursed <- weekdays(data$DisbursalDate)

In [10]:
data$total_loans <- ifelse(is.nan( data$PRI.NO.OF.ACCTS+data$SEC.NO.OF.ACCTS),0,ifelse(is.infinite(data$PRI.NO.OF.ACCTS+data$SEC.NO.OF.ACCTS),0,data$PRI.NO.OF.ACCTS+data$SEC.NO.OF.ACCTS))
data$total_active_loans<- ifelse(is.nan( data$PRI.ACTIVE.ACCTS+data$SEC.ACTIVE.ACCTS),0,ifelse(is.infinite(data$PRI.ACTIVE.ACCTS+data$SEC.ACTIVE.ACCTS),0,data$PRI.ACTIVE.ACCTS+data$SEC.ACTIVE.ACCTS))
data$total_default_loans <- ifelse(is.nan( data$PRI.OVERDUE.ACCTS+data$SEC.OVERDUE.ACCTS),0,ifelse(is.infinite(data$PRI.OVERDUE.ACCTS+data$SEC.OVERDUE.ACCTS),0,data$PRI.OVERDUE.ACCTS+data$SEC.OVERDUE.ACCTS))
data$total_cur_bal <- ifelse(is.nan( data$PRI.CURRENT.BALANCE+data$SEC.CURRENT.BALANCE),0,ifelse(is.infinite(data$PRI.CURRENT.BALANCE+data$SEC.CURRENT.BALANCE),0,data$PRI.CURRENT.BALANCE+data$SEC.CURRENT.BALANCE))
data$total_sanc_amt <- ifelse(is.nan( data$PRI.SANCTIONED.AMOUNT+data$SEC.SANCTIONED.AMOUNT),0,ifelse(is.infinite(data$PRI.SANCTIONED.AMOUNT+data$SEC.SANCTIONED.AMOUNT),0,data$PRI.SANCTIONED.AMOUNT+data$SEC.SANCTIONED.AMOUNT))
data$total_disb_amt<- ifelse(is.nan(data$PRI.DISBURSED.AMOUNT+data$SEC.DISBURSED.AMOUNT),0,ifelse(is.infinite(data$PRI.DISBURSED.AMOUNT+data$SEC.DISBURSED.AMOUNT),0,data$PRI.DISBURSED.AMOUNT+data$SEC.DISBURSED.AMOUNT))
data$total_instal_amt <- ifelse(is.nan( data$PRIMARY.INSTAL.AMT+data$SEC.INSTAL.AMT),0,ifelse(is.infinite(data$PRIMARY.INSTAL.AMT+data$SEC.INSTAL.AMT),0,data$PRIMARY.INSTAL.AMT+data$SEC.INSTAL.AMT))
data$perc_total_pri_loans <- ifelse(is.nan( data$PRI.NO.OF.ACCTS/data$total_loans),0,ifelse(is.infinite(data$PRI.NO.OF.ACCTS/data$total_loans),0,data$PRI.NO.OF.ACCTS/data$total_loans))
data$perc_act_pri_loans <- ifelse(is.nan( data$PRI.ACTIVE.ACCTS/data$PRI.NO.OF.ACCTS),0,ifelse(is.infinite(data$PRI.ACTIVE.ACCTS/data$PRI.NO.OF.ACCTS),0,data$PRI.ACTIVE.ACCTS/data$PRI.NO.OF.ACCTS))
data$perc_act_prifromtot_loans <- ifelse(is.nan( data$PRI.ACTIVE.ACCTS/data$total_active_loans),0,ifelse(is.infinite(data$PRI.ACTIVE.ACCTS/data$total_active_loans),0,data$PRI.ACTIVE.ACCTS/data$total_active_loans))
data$perc_default_pri_loans<- ifelse(is.nan( data$PRI.OVERDUE.ACCTS/data$PRI.NO.OF.ACCTS),0,ifelse(is.infinite(data$PRI.OVERDUE.ACCTS/data$PRI.NO.OF.ACCTS),0,data$PRI.OVERDUE.ACCTS/data$PRI.NO.OF.ACCTS))
data$perc_default_prifromact_loans<- ifelse(is.nan( data$PRI.OVERDUE.ACCTS/data$PRI.ACTIVE.ACCTS),0,ifelse(is.infinite(data$PRI.OVERDUE.ACCTS/data$PRI.ACTIVE.ACCTS),0,data$PRI.OVERDUE.ACCTS/data$PRI.ACTIVE.ACCTS))
data$perc_default_prifromtot_loans<- ifelse(is.nan( data$PRI.OVERDUE.ACCTS/data$total_default_loans),0,ifelse(is.infinite(data$PRI.OVERDUE.ACCTS/data$total_default_loans),0,data$PRI.OVERDUE.ACCTS/data$total_default_loans))
data$perc_pri_cur_bal<- ifelse(is.nan( data$PRI.CURRENT.BALANCE/data$total_cur_bal),0,ifelse(is.infinite(data$PRI.CURRENT.BALANCE/data$total_cur_bal),0,data$PRI.CURRENT.BALANCE/data$total_cur_bal))
#data$perc_pri_sanc_amt <- ifelse(is.nan( data$PRI.SANCTIONED.AMOUNT/data$PRI.CURRENT.BALANCE),0,ifelse(is.infinite(data$PRI.SANCTIONED.AMOUNT/data$PRI.CURRENT.BALANCE),0,data$PRI.SANCTIONED.AMOUNT/data$PRI.CURRENT.BALANCE))
data$perc_pri_sancfromtot_amt <- ifelse(is.nan( data$PRI.SANCTIONED.AMOUNT/data$total_sanc_amt),0,ifelse(is.infinite(data$PRI.SANCTIONED.AMOUNT/data$total_sanc_amt),0,data$PRI.SANCTIONED.AMOUNT/data$total_sanc_amt))
#data$perc_pri_disb_amt <- ifelse(is.nan( data$PRI.DISBURSED.AMOUNT/data$PRI.CURRENT.BALANCE),0,ifelse(is.infinite(data$PRI.DISBURSED.AMOUNT/data$PRI.CURRENT.BALANCE),0,data$PRI.DISBURSED.AMOUNT/data$PRI.CURRENT.BALANCE))
data$perc_pri_disbfromtot_amt <- ifelse(is.nan( data$PRI.DISBURSED.AMOUNT/data$total_disb_amt),0,ifelse(is.infinite(data$PRI.DISBURSED.AMOUNT/data$total_disb_amt),0,data$PRI.DISBURSED.AMOUNT/data$total_disb_amt))
data$perc_pri_disb_fromsanc_amt <- ifelse(is.nan( data$PRI.DISBURSED.AMOUNT/data$PRI.SANCTIONED.AMOUNT),0,ifelse(is.infinite(data$PRI.DISBURSED.AMOUNT/data$PRI.SANCTIONED.AMOUNT),0,data$PRI.DISBURSED.AMOUNT/data$PRI.SANCTIONED.AMOUNT))
data$perc_pri_disb_fromsanctot_amt <- ifelse(is.nan( data$PRI.DISBURSED.AMOUNT/data$total_sanc_amt),0,ifelse(is.infinite(data$PRI.DISBURSED.AMOUNT/data$total_sanc_amt),0,data$PRI.DISBURSED.AMOUNT/data$total_sanc_amt))
data$perc_pri_instal_amt <- ifelse(is.nan( data$PRIMARY.INSTAL.AMT/data$total_instal_amt),0,ifelse(is.infinite(data$PRIMARY.INSTAL.AMT/data$total_instal_amt),0,data$PRIMARY.INSTAL.AMT/data$total_instal_amt))
data$perc_total_sec_loans <- ifelse(is.nan( data$SEC.NO.OF.ACCTS/data$total_loans),0,ifelse(is.infinite(data$SEC.NO.OF.ACCTS/data$total_loans),0,data$SEC.NO.OF.ACCTS/data$total_loans))
data$perc_act_sec_loans <- ifelse(is.nan( data$SEC.ACTIVE.ACCTS/data$SEC.NO.OF.ACCTS),0,ifelse(is.infinite(data$SEC.ACTIVE.ACCTS/data$SEC.NO.OF.ACCTS),0,data$SEC.ACTIVE.ACCTS/data$SEC.NO.OF.ACCTS))
data$perc_act_secfromtot_loans <- ifelse(is.nan( data$SEC.ACTIVE.ACCTS/data$total_active_loans),0,ifelse(is.infinite(data$SEC.ACTIVE.ACCTS/data$total_active_loans),0,data$SEC.ACTIVE.ACCTS/data$total_active_loans))
data$perc_default_sec_loans<- ifelse(is.nan( data$SEC.OVERDUE.ACCTS/data$SEC.NO.OF.ACCTS),0,ifelse(is.infinite(data$SEC.OVERDUE.ACCTS/data$SEC.NO.OF.ACCTS),0,data$SEC.OVERDUE.ACCTS/data$SEC.NO.OF.ACCTS))
data$perc_default_secfromact_loans<- ifelse(is.nan( data$SEC.OVERDUE.ACCTS/data$SEC.ACTIVE.ACCTS),0,ifelse(is.infinite(data$SEC.OVERDUE.ACCTS/data$SEC.ACTIVE.ACCTS),0,data$SEC.OVERDUE.ACCTS/data$SEC.ACTIVE.ACCTS))
data$perc_default_secfromtot_loans<- ifelse(is.nan( data$SEC.OVERDUE.ACCTS/data$total_default_loans),0,ifelse(is.infinite(data$SEC.OVERDUE.ACCTS/data$total_default_loans),0,data$SEC.OVERDUE.ACCTS/data$total_default_loans))
data$perc_sec_cur_bal<- ifelse(is.nan( data$SEC.CURRENT.BALANCE/data$total_cur_bal),0,ifelse(is.infinite(data$SEC.CURRENT.BALANCE/data$total_cur_bal),0,data$SEC.CURRENT.BALANCE/data$total_cur_bal))
#data$perc_sec_sanc_amt <- ifelse(is.nan( data$SEC.SANCTIONED.AMOUNT/data$SEC.CURRENT.BALANCE),0,ifelse(is.infinite(data$SEC.SANCTIONED.AMOUNT/data$SEC.CURRENT.BALANCE),0,data$SEC.SANCTIONED.AMOUNT/data$SEC.CURRENT.BALANCE))
data$perc_sec_sancfromtot_amt <- ifelse(is.nan( data$SEC.SANCTIONED.AMOUNT/data$total_sanc_amt),0,ifelse(is.infinite(data$SEC.SANCTIONED.AMOUNT/data$total_sanc_amt),0,data$SEC.SANCTIONED.AMOUNT/data$total_sanc_amt))
#data$perc_sec_disb_amt <- ifelse(is.nan( data$SEC.DISBURSED.AMOUNT/data$SEC.CURRENT.BALANCE),0,ifelse(is.infinite(data$SEC.DISBURSED.AMOUNT/data$SEC.CURRENT.BALANCE),0,data$SEC.DISBURSED.AMOUNT/data$SEC.CURRENT.BALANCE))
data$perc_sec_disbfromtot_amt <- ifelse(is.nan( data$SEC.DISBURSED.AMOUNT/data$total_disb_amt),0,ifelse(is.infinite(data$SEC.DISBURSED.AMOUNT/data$total_disb_amt),0,data$SEC.DISBURSED.AMOUNT/data$total_disb_amt))
data$perc_sec_disb_fromsanc_amt <- ifelse(is.nan( data$SEC.DISBURSED.AMOUNT/data$SEC.SANCTIONED.AMOUNT),0,ifelse(is.infinite(data$SEC.DISBURSED.AMOUNT/data$SEC.SANCTIONED.AMOUNT),0,data$SEC.DISBURSED.AMOUNT/data$SEC.SANCTIONED.AMOUNT))
data$perc_sec_disb_fromsanctot_amt <- ifelse(is.nan( data$SEC.DISBURSED.AMOUNT/data$total_sanc_amt),0,ifelse(is.infinite(data$SEC.DISBURSED.AMOUNT/data$total_sanc_amt),0,data$SEC.DISBURSED.AMOUNT/data$total_sanc_amt))
data$perc_sec_instal_amt <- ifelse(is.nan( data$SEC.INSTAL.AMT/data$total_instal_amt),0,ifelse(is.infinite(data$SEC.INSTAL.AMT/data$total_instal_amt),0,data$SEC.INSTAL.AMT/data$total_instal_amt))
data$perc_active_loans <- ifelse(is.nan( data$total_active_loans/data$total_loans),0,ifelse(is.infinite(data$total_active_loans/data$total_loans),0,data$total_active_loans/data$total_loans))
data$perc_default_loans <- ifelse(is.nan( data$total_default_loans/data$total_loans),0,ifelse(is.infinite(data$total_default_loans/data$total_loans),0,data$total_default_loans/data$total_loans))
data$perc_sanc_amt <- ifelse(is.nan( data$total_sanc_amt/data$total_cur_bal),0,ifelse(is.infinite(data$total_sanc_amt/data$total_cur_bal),0,data$total_sanc_amt/data$total_cur_bal))
data$perc_disb_amt <- ifelse(is.nan( data$total_disb_amt/data$total_cur_bal),0,ifelse(is.infinite(data$total_disb_amt/data$total_cur_bal),0,data$total_disb_amt/data$total_cur_bal))
data$perc_disbfromsanc_amt <- ifelse(is.nan( data$total_disb_amt/data$total_sanc_amt),0,ifelse(is.infinite(data$total_disb_amt/data$total_sanc_amt),0,data$total_disb_amt/data$total_sanc_amt))
data$total_closed_loans <- ifelse(is.nan( (data$total_loans-data$total_active_loans)),0,ifelse(is.infinite(data$total_loans-data$total_active_loans),0,data$total_loans-data$total_active_loans))
data$perc_total_closed_loans <- ifelse(is.nan( data$total_closed_loans/data$total_loans),0,ifelse(is.infinite(data$total_closed_loans/data$total_loans),0,data$total_closed_loans/data$total_loans))
data$perc_pri_closed_loans <- ifelse(is.nan( (data$PRI.ACTIVE.ACCTS-data$PRI.NO.OF.ACCTS)/data$PRI.NO.OF.ACCTS),0,ifelse(is.infinite((data$PRI.ACTIVE.ACCTS-data$PRI.NO.OF.ACCTS)/data$PRI.NO.OF.ACCTS),0,(data$PRI.ACTIVE.ACCTS-data$PRI.NO.OF.ACCTS)/data$PRI.NO.OF.ACCTS))
data$perc_sec_closed_loans <- ifelse(is.nan( (data$SEC.ACTIVE.ACCTS-data$SEC.NO.OF.ACCTS)/data$SEC.NO.OF.ACCTS),0,ifelse(is.infinite((data$SEC.ACTIVE.ACCTS-data$SEC.NO.OF.ACCTS)/data$SEC.NO.OF.ACCTS),0,(data$SEC.ACTIVE.ACCTS-data$SEC.NO.OF.ACCTS)/data$SEC.NO.OF.ACCTS))
data$perc_prifromtot_closed_loans <- ifelse(is.nan( (data$PRI.ACTIVE.ACCTS-data$PRI.NO.OF.ACCTS)/data$total_closed_loans),0,ifelse(is.infinite((data$PRI.ACTIVE.ACCTS-data$PRI.NO.OF.ACCTS)/data$total_closed_loans),0,(data$PRI.ACTIVE.ACCTS-data$PRI.NO.OF.ACCTS)/data$total_closed_loans))
data$perc_secfromtot_closed_loans <- ifelse(is.nan( (data$SEC.ACTIVE.ACCTS-data$SEC.NO.OF.ACCTS)/data$total_closed_loans),0,ifelse(is.infinite((data$SEC.ACTIVE.ACCTS-data$SEC.NO.OF.ACCTS)/data$total_closed_loans),0,(data$SEC.ACTIVE.ACCTS-data$SEC.NO.OF.ACCTS)/data$total_closed_loans))
data$total_emi_amt <- ifelse(is.nan( data$PRIMARY.INSTAL.AMT+data$SEC.INSTAL.AMT),0,ifelse(is.infinite(data$PRIMARY.INSTAL.AMT+data$SEC.INSTAL.AMT),0,data$PRIMARY.INSTAL.AMT+data$SEC.INSTAL.AMT))
data$perc_pri_emi_amt <- ifelse(is.nan( data$PRIMARY.INSTAL.AMT/data$total_emi_amt),0,ifelse(is.infinite(data$PRIMARY.INSTAL.AMT/data$total_emi_amt),0,data$PRIMARY.INSTAL.AMT/data$total_emi_amt))
data$perc_sec_emi_amt <- ifelse(is.nan( data$SEC.INSTAL.AMT/data$total_emi_amt),0,ifelse(is.infinite(data$SEC.INSTAL.AMT/data$total_emi_amt),0,data$SEC.INSTAL.AMT/data$total_emi_amt))
data$perc_default_before_6mnths <- ifelse(is.nan( data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$NEW.ACCTS.IN.LAST.SIX.MONTHS),0,ifelse(is.infinite(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$NEW.ACCTS.IN.LAST.SIX.MONTHS),0,data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$NEW.ACCTS.IN.LAST.SIX.MONTHS))
data$perc_tot_bal_from_tot_sanc <- ifelse(is.nan( data$total_cur_bal/data$total_sanc_amt),0,ifelse(is.infinite(data$total_cur_bal/data$total_sanc_amt),0,data$total_cur_bal/data$total_sanc_amt))
data$perc_tot_bal_from_tot_disb <- ifelse(is.nan( data$total_cur_bal/data$total_disb_amt),0,ifelse(is.infinite(data$total_cur_bal/data$total_disb_amt),0,data$total_cur_bal/data$total_disb_amt))
data$perc_pri_bal_from_tot_sanc <- ifelse(is.nan( data$PRI.CURRENT.BALANCE/data$total_sanc_amt),0,ifelse(is.infinite(data$PRI.CURRENT.BALANCE/data$total_sanc_amt),0,data$PRI.CURRENT.BALANCE/data$total_sanc_amt))
data$perc_pri_bal_from_tot_disb <- ifelse(is.nan( data$PRI.CURRENT.BALANCE/data$total_disb_amt),0,ifelse(is.infinite(data$PRI.CURRENT.BALANCE/data$total_disb_amt),0,data$PRI.CURRENT.BALANCE/data$total_disb_amt))
data$perc_pri_bal_from_pri_sanc <- ifelse(is.nan( data$PRI.CURRENT.BALANCE/data$PRI.SANCTIONED.AMOUNT),0,ifelse(is.infinite(data$PRI.CURRENT.BALANCE/data$PRI.SANCTIONED.AMOUNT),0,data$PRI.CURRENT.BALANCE/data$PRI.SANCTIONED.AMOUNT))
data$perc_pri_bal_from_pri_disb <- ifelse(is.nan( data$PRI.CURRENT.BALANCE/data$PRI.DISBURSED.AMOUNT),0,ifelse(is.infinite(data$PRI.CURRENT.BALANCE/data$PRI.DISBURSED.AMOUNT),0,data$PRI.CURRENT.BALANCE/data$PRI.DISBURSED.AMOUNT))
data$perc_sec_bal_from_tot_sanc <- ifelse(is.nan( data$SEC.CURRENT.BALANCE/data$total_sanc_amt),0,ifelse(is.infinite(data$SEC.CURRENT.BALANCE/data$total_sanc_amt),0,data$SEC.CURRENT.BALANCE/data$total_sanc_amt))
data$perc_sec_bal_from_tot_disb <- ifelse(is.nan( data$SEC.CURRENT.BALANCE/data$total_disb_amt),0,ifelse(is.infinite(data$SEC.CURRENT.BALANCE/data$total_disb_amt),0,data$SEC.CURRENT.BALANCE/data$total_disb_amt))
data$perc_sec_bal_from_sec_sanc <- ifelse(is.nan( data$SEC.CURRENT.BALANCE/data$SEC.SANCTIONED.AMOUNT),0,ifelse(is.infinite(data$SEC.CURRENT.BALANCE/data$SEC.SANCTIONED.AMOUNT),0,data$SEC.CURRENT.BALANCE/data$SEC.SANCTIONED.AMOUNT))
data$perc_sec_bal_from_sec_disb <- ifelse(is.nan( data$SEC.CURRENT.BALANCE/data$SEC.DISBURSED.AMOUNT),0,ifelse(is.infinite(data$SEC.CURRENT.BALANCE/data$SEC.DISBURSED.AMOUNT),0,data$SEC.CURRENT.BALANCE/data$SEC.DISBURSED.AMOUNT))
data$ratio_disb_asset <- ifelse(is.nan( round(data$disbursed_amount/data$asset_cost,2)),0,ifelse(is.infinite(round(data$disbursed_amount/data$asset_cost,2)),0,round(data$disbursed_amount/data$asset_cost,2)))
data$diff_ltv_disb <- ifelse(is.nan( round(data$ltv-data$ratio_disb_asset,2)),0,ifelse(is.infinite(round(data$ltv-data$ratio_disb_asset,2)),0,round(data$ltv-data$ratio_disb_asset,2)))
data$perc_disb_ltv <- ifelse(is.nan( round(data$ratio_disb_asset/data$ltv,2)),0,ifelse(is.infinite(round(data$ratio_disb_asset/data$ltv,2)),0,round(data$ratio_disb_asset/data$ltv,2)))
data$perc_disb_pri_curr_bal <- ifelse(is.nan( round(data$disbursed_amount/data$PRI.CURRENT.BALANCE,2)),0,ifelse(is.infinite(round(data$disbursed_amount/data$PRI.CURRENT.BALANCE,2)),0,round(data$disbursed_amount/data$PRI.CURRENT.BALANCE,2)))
data$perc_disb_pri_sanc_amt <- ifelse(is.nan( round(data$disbursed_amount/data$PRI.SANCTIONED.AMOUNT,2)),0,ifelse(is.infinite(round(data$disbursed_amount/data$PRI.SANCTIONED.AMOUNT,2)),0,round(data$disbursed_amount/data$PRI.SANCTIONED.AMOUNT,2)))
data$perc_disb_pri_disb_amt <- ifelse(is.nan( round(data$disbursed_amount/data$PRI.DISBURSED.AMOUNT,2)),0,ifelse(is.infinite(round(data$disbursed_amount/data$PRI.DISBURSED.AMOUNT,2)),0,round(data$disbursed_amount/data$PRI.DISBURSED.AMOUNT,2)))
data$perc_disb_sec_curr_bal <- ifelse(is.nan( round(data$disbursed_amount/data$SEC.CURRENT.BALANCE,2)),0,ifelse(is.infinite(round(data$disbursed_amount/data$SEC.CURRENT.BALANCE,2)),0,round(data$disbursed_amount/data$SEC.CURRENT.BALANCE,2)))
data$perc_disb_sec_sanc_amt <- ifelse(is.nan( round(data$disbursed_amount/data$SEC.SANCTIONED.AMOUNT,2)),0,ifelse(is.infinite(round(data$disbursed_amount/data$SEC.SANCTIONED.AMOUNT,2)),0,round(data$disbursed_amount/data$SEC.SANCTIONED.AMOUNT,2)))
data$perc_disb_sec_disb_amt <- ifelse(is.nan( round(data$disbursed_amount/data$SEC.DISBURSED.AMOUNT,2)),0,ifelse(is.infinite(round(data$disbursed_amount/data$SEC.DISBURSED.AMOUNT,2)),0,round(data$disbursed_amount/data$SEC.DISBURSED.AMOUNT,2)))
data$perc_disb_tot_curr_bal <- ifelse(is.nan( round(data$disbursed_amount/data$total_cur_bal,2)),0,ifelse(is.infinite(round(data$disbursed_amount/data$total_cur_bal,2)),0,round(data$disbursed_amount/data$total_cur_bal,2)))
data$perc_disb_tot_sanc_amt <- ifelse(is.nan( round(data$disbursed_amount/data$total_sanc_amt,2)),0,ifelse(is.infinite(round(data$disbursed_amount/data$total_sanc_amt,2)),0,round(data$disbursed_amount/data$total_sanc_amt,2)))
data$perc_disb_tot_disb_amt <- ifelse(is.nan( round(data$disbursed_amount/data$total_disb_amt,2)),0,ifelse(is.infinite(round(data$disbursed_amount/data$total_disb_amt,2)),0,round(data$disbursed_amount/data$total_disb_amt,2)))
data$perc_del_tot_loans <- ifelse(is.nan( round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$total_loans,2)),0,ifelse(is.infinite(round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$total_loans,2)),0,round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$total_loans,2)))
data$perc_del_pri_loans <- ifelse(is.nan( round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.NO.OF.ACCTS,2)),0,ifelse(is.infinite(round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.NO.OF.ACCTS,2)),0,round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.NO.OF.ACCTS,2)))
data$perc_del_sec_loans <- ifelse(is.nan( round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.NO.OF.ACCTS,2)),0,ifelse(is.infinite(round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.NO.OF.ACCTS,2)),0,round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.NO.OF.ACCTS,2)))
data$perc_del_tot_act_loans <- ifelse(is.nan( round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$total_active_loans,2)),0,ifelse(is.infinite(round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$total_active_loans,2)),0,round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$total_active_loans,2)))
data$perc_del_pri_act_loans <- ifelse(is.nan( round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.ACTIVE.ACCTS,2)),0,ifelse(is.infinite(round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.ACTIVE.ACCTS,2)),0,round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.ACTIVE.ACCTS,2)))
data$perc_del_sec_act_loans <- ifelse(is.nan( round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.ACTIVE.ACCTS,2)),0,ifelse(is.infinite(round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.ACTIVE.ACCTS,2)),0,round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.ACTIVE.ACCTS,2)))
data$perc_del_tot_def_loans <- ifelse(is.nan( round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$total_default_loans,2)),0,ifelse(is.infinite(round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$total_default_loans,2)),0,round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$total_default_loans,2)))
data$perc_del_pri_def_loans <- ifelse(is.nan( round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.OVERDUE.ACCTS,2)),0,ifelse(is.infinite(round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.OVERDUE.ACCTS,2)),0,round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.OVERDUE.ACCTS,2)))
data$perc_del_sec_def_loans <- ifelse(is.nan( round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.OVERDUE.ACCTS,2)),0,ifelse(is.infinite(round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.OVERDUE.ACCTS,2)),0,round(data$DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.OVERDUE.ACCTS,2)))
data$perc_new_last6mnths_tot <- ifelse(is.nan( round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$total_loans,2)),0,ifelse(is.infinite(round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$total_loans,2)),0,round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$total_loans,2)))
data$perc_new_last6mnths_tot_act <- ifelse(is.nan( round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$total_active_loans,2)),0,ifelse(is.infinite(round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$total_active_loans,2)),0,round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$total_active_loans,2)))
data$perc_new_last6mnths_pri <- ifelse(is.nan( round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.NO.OF.ACCTS,2)),0,ifelse(is.infinite(round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.NO.OF.ACCTS,2)),0,round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.NO.OF.ACCTS,2)))
data$perc_new_last6mnths_pri_act <- ifelse(is.nan( round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.ACTIVE.ACCTS,2)),0,ifelse(is.infinite(round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.ACTIVE.ACCTS,2)),0,round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$PRI.ACTIVE.ACCTS,2)))
data$perc_new_last6mnths_sec <- ifelse(is.nan( round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.NO.OF.ACCTS,2)),0,ifelse(is.infinite(round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.NO.OF.ACCTS,2)),0,round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.NO.OF.ACCTS,2)))
data$perc_new_last6mnths_sec_act <- ifelse(is.nan( round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.ACTIVE.ACCTS,2)),0,ifelse(is.infinite(round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.ACTIVE.ACCTS,2)),0,round(data$NEW.ACCTS.IN.LAST.SIX.MONTHS/data$SEC.ACTIVE.ACCTS,2)))

#data$total_loan_tenure <- ifelse(is.nan( data$AVERAGE.ACCT.AGE*data$total_loans),0,ifelse(is.infinite(data$AVERAGE.ACCT.AGE*data$total_loans),0,data$AVERAGE.ACCT.AGE*data$total_loans))


In [11]:
dummy = data$CREDIT.HISTORY.LENGTH
dummy = gsub("\\s+", " ", str_trim(dummy))
out <- strsplit(as.character(dummy),' ') 
out = as.data.frame(do.call(rbind, out))
names(out) <- c("year","month")
regexp <- "[[:digit:]]+"
out$yr_frst_loan = as.numeric(paste0(str_extract(out$year, regexp),".", str_extract(out$month, regexp), sep=""))
out$mnth_frst_loan = (as.numeric(str_extract(out$year, regexp))*12)+(as.numeric(str_extract(out$month, regexp)))
data <- cbind(data,out[c(3,4)])
data$CREDIT.HISTORY.LENGTH=NULL

data$enq_since_frst_loan_yr = ifelse(is.nan(round(data$NO.OF_INQUIRIES/data$yr_frst_loan,0)),0,ifelse(is.infinite(round(data$NO.OF_INQUIRIES/data$yr_frst_loan,0)),0,round(data$NO.OF_INQUIRIES/data$yr_frst_loan,0)))
data$enq_since_frst_loan_mnth = ifelse(is.nan(round(data$NO.OF_INQUIRIES/data$mnth_frst_loan,0)),0,ifelse(is.infinite(round(data$NO.OF_INQUIRIES/data$mnth_frst_loan,0)),0,round(data$NO.OF_INQUIRIES/data$mnth_frst_loan,0)))
data$avg_enq_per_loan = ifelse(is.nan(round(data$NO.OF_INQUIRIES/data$total_loans,0)),0,ifelse(is.infinite(round(data$NO.OF_INQUIRIES/data$total_loans,0)),0,round(data$NO.OF_INQUIRIES/data$total_loans,0)))

In [12]:
dummy = data$AVERAGE.ACCT.AGE
dummy = gsub("\\s+", " ", str_trim(dummy))
out <- strsplit(as.character(dummy),' ') 
out = as.data.frame(do.call(rbind, out))
names(out) <- c("year","month")
out$avg_acnt_age_years = as.numeric(paste0(str_extract(out$year, regexp),".", str_extract(out$month, regexp), sep=""))
out$avg_acnt_age_months = (as.numeric(str_extract(out$year, regexp))*12)+(as.numeric(str_extract(out$month, regexp)))
data <- cbind(data,out[c(3,4)])
data$AVERAGE.ACCT.AGE=NULL

data$total_loan_tenure_yr <- data$avg_acnt_age_years*data$total_loans
data$total_loan_tenure_mnth <- data$avg_acnt_age_months*data$total_loans

In [13]:
names(data)

[1] "UniqueID"                           
  [2] "disbursed_amount"                   
  [3] "asset_cost"                         
  [4] "ltv"                                
  [5] "branch_id"                          
  [6] "supplier_id"                        
  [7] "manufacturer_id"                    
  [8] "Current_pincode_ID"                 
  [9] "Date.of.Birth"                      
 [10] "Employment.Type"                    
 [11] "DisbursalDate"                      
 [12] "State_ID"                           
 [13] "Employee_code_ID"                   
 [14] "MobileNo_Avl_Flag"                  
 [15] "Aadhar_flag"                        
 [16] "PAN_flag"                           
 [17] "VoterID_flag"                       
 [18] "Driving_flag"                       
 [19] "Passport_flag"                      
 [20] "PERFORM_CNS.SCORE"                  
 [21] "PERFORM_CNS.SCORE.DESCRIPTION"      
 [22] "PRI.NO.OF.ACCTS"                    
 [23] "PRI.ACTIVE.ACCTS"                   
 [24] "PRI.OVERDUE.ACCTS"                  
 [25] "PRI.CURRENT.BALANCE"                
 [26] "PRI.SANCTIONED.AMOUNT"              
 [27] "PRI.DISBURSED.AMOUNT"               
 [28] "SEC.NO.OF.ACCTS"                    
 [29] "SEC.ACTIVE.ACCTS"                   
 [30] "SEC.OVERDUE.ACCTS"                  
 [31] "SEC.CURRENT.BALANCE"                
 [32] "SEC.SANCTIONED.AMOUNT"              
 [33] "SEC.DISBURSED.AMOUNT"               
 [34] "PRIMARY.INSTAL.AMT"                 
 [35] "SEC.INSTAL.AMT"                     
 [36] "NEW.ACCTS.IN.LAST.SIX.MONTHS"       
 [37] "DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS"
 [38] "NO.OF_INQUIRIES"                    
 [39] "flag"                               
 [40] "Age"                                
 [41] "Month_Disbursed"                    
 [42] "Day_Disbursed"                      
 [43] "Week_Disbursed"                     
 [44] "Weekday_Disbursed"                  
 [45] "total_loans"                        
 [46] "total_active_loans"                 
 [47] "total_default_loans"                
 [48] "total_cur_bal"                      
 [49] "total_sanc_amt"                     
 [50] "total_disb_amt"                     
 [51] "total_instal_amt"                   
 [52] "perc_total_pri_loans"               
 [53] "perc_act_pri_loans"                 
 [54] "perc_act_prifromtot_loans"          
 [55] "perc_default_pri_loans"             
 [56] "perc_default_prifromact_loans"      
 [57] "perc_default_prifromtot_loans"      
 [58] "perc_pri_cur_bal"                   
 [59] "perc_pri_sancfromtot_amt"           
 [60] "perc_pri_disbfromtot_amt"           
 [61] "perc_pri_disb_fromsanc_amt"         
 [62] "perc_pri_disb_fromsanctot_amt"      
 [63] "perc_pri_instal_amt"                
 [64] "perc_total_sec_loans"               
 [65] "perc_act_sec_loans"                 
 [66] "perc_act_secfromtot_loans"          
 [67] "perc_default_sec_loans"             
 [68] "perc_default_secfromact_loans"      
 [69] "perc_default_secfromtot_loans"      
 [70] "perc_sec_cur_bal"                   
 [71] "perc_sec_sancfromtot_amt"           
 [72] "perc_sec_disbfromtot_amt"           
 [73] "perc_sec_disb_fromsanc_amt"         
 [74] "perc_sec_disb_fromsanctot_amt"      
 [75] "perc_sec_instal_amt"                
 [76] "perc_active_loans"                  
 [77] "perc_default_loans"                 
 [78] "perc_sanc_amt"                      
 [79] "perc_disb_amt"                      
 [80] "perc_disbfromsanc_amt"              
 [81] "total_closed_loans"                 
 [82] "perc_total_closed_loans"            
 [83] "perc_pri_closed_loans"              
 [84] "perc_sec_closed_loans"              
 [85] "perc_prifromtot_closed_loans"       
 [86] "perc_secfromtot_closed_loans"       
 [87] "total_emi_amt"                      
 [88] "perc_pri_emi_amt"                   
 [89] "perc_sec_emi_amt"                   
 [90] "perc_default_before_6mnths"         
 [91] "perc_tot_bal_from_tot_sanc"        

In [14]:
data$avg_loan_tot_amt <- ifelse(is.nan( round(data$avg_acnt_age_months*data$total_emi_amt,2)),0,ifelse(is.infinite(round(data$avg_acnt_age_months*data$total_emi_amt,2)),0,round(data$avg_acnt_age_months*data$total_emi_amt,2)))
data$avg_loan_pri_amt <- ifelse(is.nan( round(data$avg_acnt_age_months*data$PRIMARY.INSTAL.AMT,2)),0,ifelse(is.infinite(round(data$avg_acnt_age_months*data$PRIMARY.INSTAL.AMT,2)),0,round(data$avg_acnt_age_months*data$PRIMARY.INSTAL.AMT,2)))
data$avg_loan_sec_amt <- ifelse(is.nan( round(data$avg_acnt_age_months*data$SEC.INSTAL.AMT,2)),0,ifelse(is.infinite(round(data$avg_acnt_age_months*data$SEC.INSTAL.AMT,2)),0,round(data$avg_acnt_age_months*data$SEC.INSTAL.AMT,2)))

In [15]:
data$diff_disb_tot <- ifelse(is.nan( round(data$disbursed_amount-data$avg_loan_tot_amt,2)),0,ifelse(is.infinite(round(data$disbursed_amount*data$avg_loan_tot_amt,2)),0,round(data$disbursed_amount*data$avg_loan_tot_amt,2)))
data$diff_disb_pri <- ifelse(is.nan( round(data$disbursed_amount-data$avg_loan_pri_amt,2)),0,ifelse(is.infinite(round(data$disbursed_amount*data$avg_loan_pri_amt,2)),0,round(data$disbursed_amount*data$avg_loan_pri_amt,2)))
data$diff_disb_sec <- ifelse(is.nan( round(data$disbursed_amount-data$avg_loan_sec_amt,2)),0,ifelse(is.infinite(round(data$disbursed_amount*data$avg_loan_sec_amt,2)),0,round(data$disbursed_amount*data$avg_loan_sec_amt,2)))

In [16]:
discard = c("Date.of.Birth","DisbursalDate","MobileNo_Avl_Flag")
#data = data[!names(data) %in% discard]
reduced_data = data[!names(data) %in% discard]
reduced_data$supplier_id <- as.numeric(reduced_data$supplier_id)

In [ ]:
#One-hot encoding of categorical vars
data1 = model.matrix(~.+0, data=reduced_data[!names(reduced_data) %in% c("flag","UniqueID")], with=F)

In [ ]:
data1 = as.data.frame(data1)
data1 = as.data.frame(cbind(data1,data$flag))
names(data1)[length(names(data1))] <- "flag"
data1$UniqueID = data$UniqueID

In [ ]:
#Splitting into train and test
train_data <- data1[data1$flag==0,]
train_data$flag=NULL
test_data <- data1[data1$flag==1,]
test_data$flag=NULL

label= train_df$loan_default
train_data$label = label
class(train_data$label)
#names(train_data)

prop.table(table(train_data[,'label']))

**Modeling using LightGBM**

In [ ]:
set.seed(123)
ind = sample.split(train_data$label, 0.05)
train = subset(train_data, ind==F)
hold = subset(train_data, ind==T)
names(train)[names(train)=="perc_disb_ltv"]

In [ ]:
#Defining the parameters for LightGBM model
lgb.grid = list(objective = "binary",
                metric = "auc",
                boosting_type="gbdt",
                #min_sum_hessian_in_leaf = 1,
                feature_fraction = 0.7,
                bagging_fraction = 0.7,
                bagging_freq = 5,
                #min_data = 100,
                #max_bin = 50,
                #lambda_l1 = 8,
                #lambda_l2 = 1.3,
                #min_data_in_bin=100,
                #min_gain_to_split = 10,
                #min_data_in_leaf = 30,
                is_unbalance = TRUE)

In [ ]:
#Creating dataset for LightGBM
lgb.train = lgb.Dataset(data=as.matrix(train[!names(train) %in% c("label","UniqueID")]), label=train$label)

In [ ]:
lgb.model.cv = lgb.cv(params = lgb.grid, data = lgb.train, learning_rate = 0.01, num_leaves = 25,
                num_threads = 2 , nrounds = 10000, early_stopping_rounds = 1000,
              eval_freq = 20, 
                #categorical_feature = factors, 
                  nfold = 5, stratified = TRUE, random_state=40)

In [ ]:
#Retreiving best score
lgb.model.cv$best_iter
lgb.model.cv$best_score

In [ ]:
#Training the model
lgb.model = lgb.train(params = lgb.grid, data = lgb.train,  learning_rate= 0.02,
                       num_threads = 2 , nrounds=3344,
                      #nrounds = best.iter,
                      eval_freq = 20, random_state=40)

In [ ]:
#Importance of features
tree_imp <- lgb.importance(lgb.model, percentage = TRUE)
lgb.plot.importance(tree_imp, top_n = 50, measure = "Gain")

In [ ]:
#Importance
tree_imp$rownames = rownames(tree_imp[order(-tree_imp$Gain),])
head(tree_imp)

In [ ]:
pred = predict(lgb.model, hold.train)

In [ ]:
#AUC score
auc(hold$label, pred)

In [ ]:
#Adding predictions to test file
pred = predict(lgb.model, as.matrix(test_data[!names(test_data) %in% c("rownames","UniqueID")]))
sub = as.data.frame(cbind(as.numeric(as.character(test_data$UniqueID)),pred))
names(sub) = c("UniqueID","loan_default")
write.csv(sub, "submission_23.csv", row.names=F)